In [61]:
%pip install pypdf
%pip install openai
%pip install azure-functions
%pip install adlfs
%pip install langchain==0.0.254
%pip install langdetect
%pip install azure-storage-blob 
%pip install python-docx
%pip install unstructured
%pip install azure-search-documents==11.4.0b6
%pip install openpyxl
%pip install lingua-language-detector

^C
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
^C
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [62]:

pip --trusted-host pypi.python.org --trusted-host pypi.tuna.tsinghua.edu.cn install --user lingua-language-detector -i https://pypi.tuna.tsinghua.edu.cn/simple/


Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple/
Note: you may need to restart the kernel to use updated packages.


In [16]:
## Import Library

import openai
from azure.core.credentials import AzureKeyCredential
from azure.identity import AzureDeveloperCliCredential
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient


"""from azure.search.documents.indexes.models import (
    HnswParameters,
    PrioritizedFields,
    SearchableField,
    SearchField,
    SearchFieldDataType,
    SearchIndex,
    SemanticConfiguration,
    SemanticField,
    SemanticSettings,
    SimpleField,
    VectorSearch,
    VectorSearchAlgorithmConfiguration,
)"""


from azure.search.documents.indexes.models import (
    HnswParameters,
    PrioritizedFields,
    SearchableField,
    SearchField,
    SearchFieldDataType,
    SearchIndex,
    SemanticConfiguration,
    SemanticField,
    SemanticSettings,
    SimpleField,
    VectorSearch,
    VectorSearchAlgorithmConfiguration,
)



from azure.storage.blob import BlobServiceClient

import openai
import os
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores.azuresearch import AzureSearch


In [17]:
import openai
import os
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores.azuresearch import AzureSearch

from azure.storage.blob import BlobServiceClient
from azure.core.exceptions import ResourceExistsError
import json

# helper class
class IndexUtil():
    def __init__(self,
                 search_service: str,
                 storage_service: str,
                 search_api_key:str,
                 storage_api_key: str):
        self.searchservice = search_service
        self.storage_service = storage_service
        self.search_api_key = search_api_key
        self.search_creds = AzureKeyCredential(search_api_key)
        self.storage_creds = storage_api_key

        # set up openai environment
        os.environ["OPENAI_API_TYPE"] = "azure"
        os.environ["OPENAI_API_BASE"] = "https://pwcjay.openai.azure.com/"
        os.environ["OPENAI_API_VERSION"] = "2023-05-15"
        os.environ["OPENAI_API_KEY"] = "f282a661571f45a0bdfdcd295ac808e7"


        # define embedding model for calculating the embeddings
        model: str = "text-embedding-ada-002"
        embeddings: OpenAIEmbeddings = OpenAIEmbeddings(deployment=model, chunk_size=1)
        embedding_function = embeddings.embed_query

        # define schema of the json file stored on the index
        self.fields = [
                SimpleField(
                    name="id",
                    type=SearchFieldDataType.String,
                    key=True,
                    filterable=True,
                ),
                SearchableField(
                    name="content",
                    type=SearchFieldDataType.String,
                    searchable=True,
                ),
                SearchField(
                    name="content_vector",
                    type=SearchFieldDataType.Collection(SearchFieldDataType),
                    searchable=True,
                    vector_search_dimensions=len(embedding_function("Text")),
                    vector_search_configuration="default",
                ),
                SearchableField(
                    name="metadata",
                    type=SearchFieldDataType.String,
                    searchable=True,
                ),
                # Additional field to store the title
                SearchableField(
                    name="title",
                    type=SearchFieldDataType.String,
                    searchable=True,
                ),
                # Additional field for filtering on document source
                SimpleField(
                    name="source",
                    type=SearchFieldDataType.String,
                    filterable=True,
                ),
                # Additional field for filtering on document source
                SimpleField(
                    name="page",
                    type=SearchFieldDataType.String,
                    filterable=True,
                ),
                # Additional field for filtering on document source
                SimpleField(
                    name="website_url",
                    type=SearchFieldDataType.String,
                    filterable=True,
                ),
            ]
        
        
        
        """# define schema of the json file stored on the index
        self.fields = [
                SimpleField(
                    name="id",
                    type=SearchFieldDataType.String,
                    key=True,
                    filterable=True,
                ),
                SearchableField(
                    name="content",
                    type=SearchFieldDataType.String,
                    searchable=True,
                ),
                SearchField(
                    name="content_vector",
                    type=SearchFieldDataType.Collection(SearchFieldDataType.EdmSingle),
                    searchable=True,
                    vector_search_configuration="default",
                ),
                SearchableField(
                    name="metadata",
                    type=SearchFieldDataType.String,
                    searchable=True,
                ),
                # Additional field to store the title
                SearchableField(
                    name="title",
                    type=SearchFieldDataType.String,
                    searchable=True,
                ),
                # Additional field for filtering on document source
                SimpleField(
                    name="source",
                    type=SearchFieldDataType.String,
                    filterable=True,
                ),
                # Additional field for filtering on document source
                SimpleField(
                    name="page",
                    type=SearchFieldDataType.String,
                    filterable=True,
                ),
                # Additional field for filtering on document source
                SimpleField(
                    name="website_url",
                    type=SearchFieldDataType.String,
                    filterable=True,
                ),
            ]"""
        


    # create index using langchain
    def create_search_index(self, index):
        print(f"Ensuring search index {index} exists")
        index_client = SearchIndexClient(endpoint=f"https://{self.searchservice}.search.windows.net/",
                                        credential=self.search_creds)
        if index not in index_client.list_index_names():
            # set up openai environment
            #os.environ["OPENAI_API_TYPE"] = "azure"
            #os.environ["OPENAI_API_BASE"] = "https://pwcjay.openai.azure.com/"
            #os.environ["OPENAI_API_VERSION"] = "2023-05-15"
            #os.environ["OPENAI_API_KEY"] = "f282a661571f45a0bdfdcd295ac808e7"

            model: str = "text-embedding-ada-002"
            vector_store_address: str = f"https://{self.searchservice}.search.windows.net"
            vector_store_password: str = self.search_api_key

            embeddings: OpenAIEmbeddings = OpenAIEmbeddings(deployment=model, chunk_size=1)
            embedding_function = embeddings.embed_query

            index_name: str = index
            vector_store: AzureSearch = AzureSearch(
                azure_search_endpoint=vector_store_address,
                azure_search_key=vector_store_password,
                index_name=index_name,
                embedding_function=embedding_function,
                fields=self.fields,
            )

    # get vector store by index name
    def _get_vector_store(self, index_name):

        model: str = "text-embedding-ada-002"
        embeddings: OpenAIEmbeddings = OpenAIEmbeddings(deployment=model, chunk_size=1)
        embedding_function = embeddings.embed_query

        vector_store_address: str = f"https://{self.searchservice}.search.windows.net"
        vector_store_password: str = self.search_api_key
        vector_store: AzureSearch = AzureSearch(
            azure_search_endpoint=vector_store_address,
            azure_search_key=vector_store_password,
            embedding_function=embedding_function,
            index_name=index_name,
            fields=self.fields)
        return vector_store


    # upload single document using langchain, add doc name, bu, region in metadata for searching purpose
    # is_blob to control whether it's upload from blob storage or not
    def upload_document(self, index_name, docs, doc_name,  container_name = None, blob_client=None,  is_blob=False):
        vector_store = self._get_vector_store(index_name)
        for doc in docs:
            if is_blob:
                website_url = self.get_blob_url(blob_client)
                doc_name_index = ''
                if ".docx" in doc_name:
                    doc_name_index = doc_name.replace('.docx', '')
                elif ".pdf" in doc_name:
                    doc_name_index = doc_name.replace('.pdf', '')
                print(doc_name_index)
                print(doc_name)
                #doc.metadata = {"title": doc_name_index, "source": doc_name, "website_url":website_url}
            else:
                doc_name_index = ''
                if ".docx" in doc_name:
                    doc_name_index = doc_name.replace('.docx', '')
                elif ".pdf" in doc_name:
                    doc_name_index = doc_name.replace('.pdf', '')
                #doc.metadata = {"title": doc_name_index, "source": doc_name, "website_url":"/" }
            print(doc)
            #doc.metadata['source']
        id_list = vector_store.add_documents(documents=docs)
        self._update_json( id_list, doc_name, container_name)        
        return


    # specific for starbucks demo
    # return web site url
    def get_blob_url(self, blob_client):
        filename = str(blob_client.blob_name).split('/')[-1] 
        if filename == "伙伴手册假期部分模拟内容.docx":
            blob_url = "https://sbstorage0.blob.core.windows.net/readfile/伙伴手册假期部分模拟内容.pdf"
        elif filename == "门店伙伴资源手册招募部分模拟内容.docx":
            blob_url = "https://sbstorage0.blob.core.windows.net/readfile/门店伙伴资源手册招募部分模拟内容.pdf"
        elif filename == "人力资源政策常见问题.docx":
            blob_url = "https://sbstorage0.blob.core.windows.net/readfile/人力资源政策常见问题.pdf"
        else:
            blob_url = blob_client.url
        return blob_url

    
    # return id list, then the program create a json file to keep track of the id, doc name, bu, region
    def _update_json(self, id_list, doc_name, container_name):
 
        filename = f"doc_list.json"

        blob_service_client = BlobServiceClient(account_url=f"https://{self.storage_service}.blob.core.windows.net/", credential=self.storage_creds)

        try:
            container_client = blob_service_client.create_container(name=container_name)
        except ResourceExistsError:
            print('A container with this name already exists')
            container_client = blob_service_client.get_container_client(container_name)
            
        blob_client = container_client.get_blob_client(filename)

        if blob_client.exists():
            streamdownloader = blob_client.download_blob()
            data = json.loads(streamdownloader.readall())
            for id in id_list:
                data[id] = doc_name
            json_data = json.dumps(data, ensure_ascii=False)
            blob_client.upload_blob(json_data, overwrite= True)
        else:
            data = {}
            for id in id_list:
                data[id] = doc_name
            json_data = json.dumps(data, ensure_ascii=False)
            blob_client.upload_blob(json_data)
        return


    # delete documents by id

    def delete_document(self, index_name, id):
        endpoint = f"https://{self.searchservice}.search.windows.net/"
        search_client = SearchClient(endpoint,index_name, self.search_creds)
        # [START delete_document]
        result = search_client.delete_documents(documents=[{"id": id}])

        print("Delete new document succeeded: {}".format(result[0].succeeded))
        # [END delete_document]
        return
    
    # delete documents by bu, region
    def delete_by_partition(self, index_name, container_name = None):
        # read blob storage of that bu and region to check that json file for ids
        
        filename =  f"doc_list.json"

        blob_service_client = BlobServiceClient(account_url=f"https://{self.storage_service}.blob.core.windows.net/", credential=self.storage_creds)

        try:
            container_client = blob_service_client.create_container(name=container_name)
        except ResourceExistsError:
            print('A container with this name already exists')
            container_client = blob_service_client.get_container_client(container_name)
            
        blob_client = container_client.get_blob_client(filename)

        # if file not exists, nothing to delete
        # else if 
        if blob_client.exists():
            streamdownloader = blob_client.download_blob()
            data = json.loads(streamdownloader.readall())
            id_list = []
            # loop json to get all the ids for deleting items
            for key, value in data.items():
                id_list.append(key)
            
            # loop to batch delete documents
            for id in id_list:
                self.delete_document(index_name, id)
            
            # delete json of that partition if it exists
            blob_client.delete_blob()
        return
    

    # compare to be deployed list with current deployed list (nice to have)
    def compare(to_deploy_list, deployed_list):
        to_be_deploy = []
        to_be_delete = []
        
        # create a list of to be deploy
        for item in to_deploy_list:
            if item not in deployed_list:
                to_be_deploy.append(item)

        # create a list of to be delete
        for item in deployed_list:
            if item not in to_deploy_list:
                to_be_delete.append(item)
        
        return to_be_deploy, to_be_delete

    # delete the whole index
    def delete_index(self, index_name):
        # [START delete_index]
        index_client = SearchIndexClient(endpoint=f"https://{self.searchservice}.search.windows.net/",
                                credential=self.search_creds)
        name = index_name
        index_client.delete_index(name)
        # [END delete_index]
        return

# end helper class
# End Index Helper Class


### Run after this

In [18]:
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import UnstructuredWordDocumentLoader
from langchain.text_splitter import CharacterTextSplitter

# change this path for the documents to be ingested

#rootdir = "C:/Users/Kenny CK Wong/Desktop/Project/Business Development/Sino Presentation/sino_hr_chatbot/docs"
rootdir = "C:/Users/Sunny CS Wong/Desktop/chatbot for fda"



In [19]:
# setting up credential

"""index_name = "sino-hr-chatbot"
search_service = "gptdemosearch"
search_api_key = "PcAZcXbX2hJsxMYExc2SnkMFO0D94p7Zw3Qzeu5WjYAzSeDMuR5O"""

index_name = "your-index-name"
search_service = "acs-testing-sunny"
search_api_key = "oygYftyrBXiWoDLoZatDKNSLFttn9frM6DE4XlSb7kAzSeBR01eY"

#kenny account
#storage_service = "sinodemostorage"
#storage_api_key = "RsGWcyVx6Tzo2SqwxhuAcIntx90rmijot5wK7QpXBJpN9c9jDq3YPaRYxbyrFJ1oWSjxs8Fnbs49+AStAP9yeQ=="


#eric account
#storage_service = "pwcericstorage"
#storage_api_key = "HlXhvHTLeg3CWiRiMQrENLJ05jBlILK1iMCbIw6f/b8tMW+QJ727f+YgedFnMyOCagJfMFqjrFjJ+ASt3ogcVA=="

#Sunny account blob
storage_service = "acstestingdfda"
storage_api_key = "5fDlKtDPD0jRnwIQovOKwzTSzYtpDoAEcyPKj9fdoyyHy2XiSrnmSnQ/H5wyl4XUm8eY6e4hcfk5+AStNwyDvg=="


connect_str = f"DefaultEndpointsProtocol=https;AccountName={storage_service};AccountKey={storage_api_key}"


os.environ["AZURE_COGNITIVE_SEARCH_SERVICE_NAME"] = search_service
os.environ["AZURE_COGNITIVE_SEARCH_API_KEY"] = search_api_key
os.environ["AZURE_INDEX_NAME"] = index_name

# end setting up credential



In [20]:
from pypdf import PdfReader
from langchain.schema import Document


# to be change
def get_blob_url(filename):
    if filename == "Employee-Handbook-EN.pdf":
        blob_url = "https://pwcericstorage.blob.core.windows.net/sinohrdocs/Employee-Handbook-EN.pdf"
    elif filename == "Employee-Handbook-CN.pdf":
        blob_url = "https://pwcericstorage.blob.core.windows.net/sinohrdocs/Employee-Handbook-CN.pdf"
    else:
        blob_url = "/"
    return blob_url


def load_pdf(filepath):
    reader = PdfReader(filepath)

    doc_list = []

    for page in reader.pages:
        #print(page.extract_text())
        #print(page.page_number)
        doc = Document(
            page_content = page.extract_text(),
        )
        # split by \\ in windows
        filename = filepath.split("\\")[-1]
        website_url =  get_blob_url(filename)       
        # add metadata for searching
        doc.metadata ={"source": filename, "page": str(page.page_number+1), "website_url":website_url} 
        doc_list.append(doc)
    return doc_list



In [21]:
# initialize index util
index_util = IndexUtil(search_service,storage_service, search_api_key, storage_api_key)

# create search index if not exists
index_util.create_search_index(index_name)

# clean up previously created index
# deployed container
deploy_container = "acs-testing-fda"

# delete index partition first and delete the json
index_util.delete_by_partition(index_name, deploy_container)

for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        if file.endswith((".pdf")):
            print (os.path.join(subdir, file))

            #loader = PyPDFLoader(os.path.join(subdir, file))

            #documents = loader.load()
            #text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=150)
            #docs = text_splitter.split_documents(documents)

            doc_name = file

            docs = load_pdf(os.path.join(subdir, file))

            # upload documents from local 
            index_util.upload_document(index_name, docs, doc_name, deploy_container)

Ensuring search index your-index-name exists
A container with this name already exists
Delete new document succeeded: True
Delete new document succeeded: True
Delete new document succeeded: True
Delete new document succeeded: True
Delete new document succeeded: True
Delete new document succeeded: True
Delete new document succeeded: True
Delete new document succeeded: True
Delete new document succeeded: True
Delete new document succeeded: True
Delete new document succeeded: True
Delete new document succeeded: True
Delete new document succeeded: True
Delete new document succeeded: True
Delete new document succeeded: True
Delete new document succeeded: True
Delete new document succeeded: True
Delete new document succeeded: True
Delete new document succeeded: True
Delete new document succeeded: True
Delete new document succeeded: True
Delete new document succeeded: True
Delete new document succeeded: True
Delete new document succeeded: True
Delete new document succeeded: True
Delete new do

A container with this name already exists
C:/Users/Sunny CS Wong/Desktop/chatbot for fda\CFR-2022-title21-vol1-chapI.pdf
page_content='1 Title 21—Food and \nDrugs \n(This book contains parts 1 to 99) \nPart \nCHAPTER I —Food and Drug Administration, Department of \nHealth and Human Services ............................................... 1 \nVerDate Sep<11>2014 10:27 Aug 01, 2022 Jkt 256071 PO 00000 Frm 00011 Fmt 8008 Sfmt 8008 Q:\\21\\21V1.TXT PC31kpayne on VMOFRWIN702 with $$_JOB' metadata={'source': 'CFR-2022-title21-vol1-chapI.pdf', 'page': '1', 'website_url': '/'}
page_content='VerDate Sep<11>2014 10:27 Aug 01, 2022 Jkt 256071 PO 00000 Frm 00012 Fmt 8008 Sfmt 8008 Q:\\21\\21V1.TXT PC31kpayne on VMOFRWIN702 with $$_JOB' metadata={'source': 'CFR-2022-title21-vol1-chapI.pdf', 'page': '2', 'website_url': '/'}
page_content='3 CHAPTER I—FOOD AND DRUG \nADMINISTRATION, DEPARTMENT OF HEALTH \nAND HUMAN SERVICES \nEDITORIAL NOTE: Nomenclature changes to chapter I appear at 66 FR 56035, Nov

A container with this name already exists
